In [1]:
import pandas as pd
import numpy as np

# Analyse pair_data.csv

In [2]:
pair_data = pd.read_csv('../17_data_new/pair_data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3526: DtypeWarning: Columns (3,11,12,13,14,15,16,18,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
pair_data.head()

,date_sample,pair_id,min_date_sample,cusips,cik_owner,company,owner,cik_company,file_type,date_issue,...,PERMNO_o,CUSIP_o,CUSIP6_o,Ticker_o,PERMCO_o,NAICS_o,DlyPrc_o,DlyCap_o,DlyVol_o,ShrOut_o
0,2017-12-29,-2147379927,2017-12-29,40167F101,895421,GUGGENHEIM STRATEGIC OPPORTUNITIES FUND,MORGAN STANLEY,1380936,13G,2018-02-12,...,69032,61744644,617446,MS,21224,523120,52.47,94860460.53,5804576.0,1807899.0
1,2017-12-29,-2147379927,2017-12-29,40167F101,895421,GUGGENHEIM STRATEGIC OPPORTUNITIES FUND,MORGAN STANLEY,1380936,13G,2018-02-12,...,69032,61744644,617446,MS,21224,523120,52.47,94860460.53,5804576.0,1807899.0
2,2018-01-02,-2147379927,2017-12-29,40167F101,895421,GUGGENHEIM STRATEGIC OPPORTUNITIES FUND,MORGAN STANLEY,1380936,13G,2018-02-12,...,69032,61744644,617446,MS,21224,523120,52.19,94354248.81,6644643.0,1807899.0
3,2018-01-03,-2147379927,2017-12-29,40167F101,895421,GUGGENHEIM STRATEGIC OPPORTUNITIES FUND,MORGAN STANLEY,1380936,13G,2018-02-12,...,69032,61744644,617446,MS,21224,523120,52.33,94607354.67,6955511.0,1807899.0
4,2018-01-04,-2147379927,2017-12-29,40167F101,895421,GUGGENHEIM STRATEGIC OPPORTUNITIES FUND,MORGAN STANLEY,1380936,13G,2018-02-12,...,69032,61744644,617446,MS,21224,523120,53.13,96053673.87,8678825.0,1807899.0


## 1. General Statistics

In [4]:
list_pair = set(pair_data['pair_id'])
print(f'The number of pairs is {len(list_pair)}.')

The number of pairs is 21319.


In [5]:
list_na_pair = set(pair_data[pair_data['shares_agg'].isna()]['pair_id'])
print(f'The number of pairs with \'Nan\' aggregated shares {len(list_na_pair)}.')

The number of pairs with 'Nan' aggregated shares 0.


In [6]:
list_na_pair = set(pair_data[pair_data['shares_agg']=='None']['pair_id'])
print(f'The number of pairs with \'None\' aggregated shares {len(list_na_pair)}.')

The number of pairs with 'None' aggregated shares 611.


In [7]:
pair_data[pair_data['shares_agg'].isna()].head(5)

,date_sample,pair_id,min_date_sample,cusips,cik_owner,company,owner,cik_company,file_type,date_issue,...,PERMNO_o,CUSIP_o,CUSIP6_o,Ticker_o,PERMCO_o,NAICS_o,DlyPrc_o,DlyCap_o,DlyVol_o,ShrOut_o


It might happen in rare cases that the parsing didn't work for the number of 'aggregated shares' but worked well for the number of 'sole dispositive shares' and 'shared dispositive shares'. In this case, I may replace the number of 'aggregated shares' by sum of the number of 'sole dispositive shares' and 'shared dispositive shares'.

In [8]:
# Convert to numeric, handling 'None' and commas
pair_data['sole_numeric'] = pd.to_numeric(pair_data['shares_sole_dispositive'].str.replace(',', ''), errors='coerce')
pair_data['shared_numeric'] = pd.to_numeric(pair_data['shares_shared_dispositive'].str.replace(',', ''), errors='coerce')

# Identify rows where both inputs are 'None' before filling NaNs with 0
mask_both_none = pair_data['sole_numeric'].isna() & pair_data['shared_numeric'].isna()

# Now fill NaNs with 0 for calculation
pair_data['sole_numeric'].fillna(0, inplace=True)
pair_data['shared_numeric'].fillna(0, inplace=True)

# Calculate the sum
pair_data['temp_sum'] = pair_data['sole_numeric'] + pair_data['shared_numeric']

# Apply mask_both_none to reintroduce 'None' where both were originally 'None'
pair_data.loc[mask_both_none, 'temp_sum'] = np.nan

# Create 'shares_agg_bis', copying 'shares_agg' first
pair_data['shares_agg_bis'] = pair_data['shares_agg']

# Replace 'None' in 'shares_agg_bis' with the calculated sum, format as string with commas
pair_data.loc[pair_data['shares_agg_bis'] == 'None', 'shares_agg_bis'] = pair_data['temp_sum'].apply(lambda x: '{:,.0f}'.format(x) if pd.notna(x) else 'None')

# Optionally, remove temporary columns if they are no longer needed
pair_data.drop(['sole_numeric', 'shared_numeric', 'temp_sum'], axis=1, inplace=True)

In [9]:
list_na_pair = set(pair_data[pair_data['shares_agg_bis']=='None']['pair_id'])
print(f'The number of pairs with \'None\' aggregated shares {len(list_na_pair)}.')

The number of pairs with 'None' aggregated shares 483.


Let's create columns with a numerical format for the columns 'shares_agg', 'shares_agg_bis' and 'shares_percentage'.

In [10]:
pair_data['shares_agg_num'] = pd.to_numeric(pair_data['shares_agg'].str.replace(',', ''), errors='coerce').fillna(0)
pair_data['shares_agg_bis_num'] = pd.to_numeric(pair_data['shares_agg_bis'].str.replace(',', ''), errors='coerce').fillna(0)
pair_data['shares_percentage_num'] = pd.to_numeric(pair_data['shares_percentage'], errors='coerce').fillna(0)

## 2. The Negative Stub Values

In [11]:
pair_data['stake_1'] = (pair_data['shares_agg_num']*pair_data['DlyPrc'])/1000
pair_data['stake_2'] = (pair_data['shares_agg_bis_num']*pair_data['DlyPrc'])/1000
pair_data['stake_3'] = pair_data['shares_percentage_num']/100*pair_data['DlyCap']
pair_data['stub_1'] = pair_data['DlyCap_o'] - pair_data['stake_1']
pair_data['stub_2'] = pair_data['DlyCap_o'] - pair_data['stake_2']
pair_data['stub_3'] = pair_data['DlyCap_o'] - pair_data['stake_3']

In [12]:
nsv_pair_1 = set(pair_data[pair_data['stub_1'] < 0]['pair_id'])
print(f'The number of pairs with a NSV is {len(nsv_pair_1)} (stub_1).')

The number of pairs with a NSV is 662 (stub_1).


In [13]:
nsv_pair_2 = set(pair_data[pair_data['stub_2'] < 0]['pair_id'])
print(f'The number of pairs with a NSV is {len(nsv_pair_2)} (stub_2).')

The number of pairs with a NSV is 666 (stub_2).


In [14]:
nsv_pair_3 = set(pair_data[pair_data['stub_3'] < 0]['pair_id'])
print(f'The number of pairs with a NSV is {len(nsv_pair_3)} (stub_3).')

The number of pairs with a NSV is 567 (stub_3).


In [15]:
nsv_pair = nsv_pair_1 | nsv_pair_2 | nsv_pair_3
print(f'The total number of NSV is {len(nsv_pair)}.')

The total number of NSV is 743.


In [16]:
nsv_pair_data = pair_data[pair_data.pair_id.isin(nsv_pair)]

In [17]:
# Step 1: Group by 'pair_id'
grouped = nsv_pair_data.groupby('pair_id')

def extract_rows(group):
    # First observation
    first_row = group.iloc[[0]]

    # Last observation
    last_row = group.iloc[[-1]]

    # Row with minimum value among 'stub_1', 'stub_2', 'stub_3'
    # Create a temporary column for the minimum of the three stubs
    group['min_stub'] = group[['stub_1', 'stub_2', 'stub_3']].min(axis=1)
    min_row = group.loc[group['min_stub'] == group['min_stub'].min()]

    # Drop the temporary column to avoid affecting the original dataframe
    group = group.drop(columns=['min_stub'])

    # It's possible min_row could have more than one row if the minimum value is achieved by multiple rows.
    # You can handle this case as needed, e.g., by taking the first occurrence:
    min_row = min_row.iloc[[0]]

    # Combine the rows
    return pd.concat([first_row, min_row, last_row])

nsv_presentation_data = pd.concat([extract_rows(group) for _, group in grouped])
nsv_presentation_data.reset_index(drop=True, inplace=True)

In [18]:
nsv_presentation_data

,date_sample,pair_id,min_date_sample,cusips,cik_owner,company,owner,cik_company,file_type,date_issue,...,shares_agg_num,shares_agg_bis_num,shares_percentage_num,stake_1,stake_2,stake_3,stub_1,stub_2,stub_3,min_stub
0,2001-03-28,-2143225085,2001-03-28,02364W105,732717,AMERICA MOVIL SA DE CV/,SBC COMMUNICATIONS INC,1129137,13G,2001-03-30,...,1.059890e+09,1.059890e+09,7.30,1.550619e+07,1.550619e+07,8.416509e+05,1.273057e+08,1.273057e+08,1.419703e+08,NaN
1,2011-02-02,-2143225085,2001-03-28,02364W105,732717,AMERICA MOVIL SAB DE CV/,AT&T INC.,1129137,13D,2010-08-02,...,3.621532e+09,3.621532e+09,0.00,2.118596e+08,2.118596e+08,0.000000e+00,-4.830227e+07,-4.830227e+07,1.635574e+08,-4.830227e+07
2,2023-03-15,-2143225085,2001-03-28,02364W105,732717,AMERICA MOVIL SAB DE CV/,AT&T INC.,1129137,13D,2014-06-30,...,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,1.305479e+08,1.305479e+08,1.305479e+08,NaN
3,2002-12-31,-2137979334,2002-12-31,559776109,946770,HUNGARIAN TELECOMMUNICATIONS CO LTD,DEUTSCHE TELEKOM AG,1047564,13G,2003-02-14,...,6.174781e+08,6.174781e+08,59.50,1.099111e+07,1.099111e+07,3.119050e+05,-8.761371e+06,-8.761371e+06,1.917834e+06,NaN
4,2007-04-20,-2137979334,2002-12-31,559776109,946770,HUNGARIAN TELECOMMUNICATIONS CO LTD,DEUTSCHE TELEKOM AG,1047564,13G,2003-02-14,...,6.174781e+08,6.174781e+08,59.50,1.814151e+07,1.814151e+07,2.137939e+05,-1.456577e+07,-1.456577e+07,3.361942e+06,-1.456577e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,1999-12-31,2144117528,1999-12-31,594972101,1050446,MICROSTRATEGY INC,MICROSTRATEGY INC,1050446,13G,2000-02-15,...,4.354932e+07,4.354932e+07,66.05,9.145358e+06,9.145358e+06,1.371792e+06,-7.068458e+06,-7.068458e+06,7.051076e+05,-7.068458e+06
2225,2002-07-30,2144117528,1999-12-31,594972101,1050446,MICROSTRATEGY INC,MICROSTRATEGY INC,1050446,13G,2000-02-18,...,8.698958e+06,8.698958e+06,28.00,4.175500e+03,4.175500e+03,6.357389e+03,1.852946e+04,1.852946e+04,1.634757e+04,NaN
2226,1999-02-16,2145733102,1999-02-16,903469104,937941,UBID INC,CREATIVE COMPUTERS INC,1064710,13G,1999-02-16,...,7.329883e+06,7.329883e+06,80.10,4.292563e+05,4.292563e+05,4.179553e+05,-1.465103e+05,-1.465103e+05,-1.352093e+05,NaN
2227,1999-03-19,2145733102,1999-02-16,903469104,937941,UBID INC,CREATIVE COMPUTERS INC,1064710,13G,1999-02-16,...,7.329883e+06,7.329883e+06,80.10,5.497412e+05,5.497412e+05,5.352682e+05,-2.144667e+05,-2.144667e+05,-1.999938e+05,-2.144667e+05


In [19]:
nsv_presentation_data[nsv_presentation_data.pair_id == -1227398985]

,date_sample,pair_id,min_date_sample,cusips,cik_owner,company,owner,cik_company,file_type,date_issue,...,shares_agg_num,shares_agg_bis_num,shares_percentage_num,stake_1,stake_2,stake_3,stub_1,stub_2,stub_3,min_stub
504,2016-09-07,-1227398985,2016-09-07,928563402,1571996,"VMWARE, INC.",Dell Technologies Inc,1124610,13D,2016-09-15,...,343025308.0,343025308.0,80.8,2.495852e+07,2.495852e+07,7.313956e+06,-1.425615e+07,-1.425615e+07,3.388412e+06,NaN
505,2019-05-16,-1227398985,2016-09-07,928563402,1571996,"VMWARE, INC.",Dell Technologies Inc,1124610,13D,2018-12-26,...,330678605.0,330678605.0,80.7,6.796107e+07,6.796107e+07,1.831715e+07,-5.596182e+07,-5.596182e+07,-6.317904e+06,-5.596182e+07
506,2023-11-21,-1227398985,2016-09-07,928563402,1571996,"VMWARE, INC.",Dell Technologies Inc.,1124610,13D,2021-11-02,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.865627e+07,1.865627e+07,1.865627e+07,NaN


In [20]:
nsv_presentation_data.to_csv('../17_data_new/nsv_presentation.csv')

In [21]:
nsv_presentation_data[nsv_presentation_data.pair_id == 1289966569]

,date_sample,pair_id,min_date_sample,cusips,cik_owner,company,owner,cik_company,file_type,date_issue,...,shares_agg_num,shares_agg_bis_num,shares_percentage_num,stake_1,stake_2,stake_3,stub_1,stub_2,stub_3,min_stub
1755,2019-03-13,1289966569,2019-03-12,03676B102,1433270,Antero Midstream Corp,ANTERO RESOURCES Corp,1623925,13D,2019-03-14,...,158419937.0,158419937.0,31.30,1.986586e+06,1.986586e+06,7.309800e+05,5.844768e+05,5.844768e+05,1.840083e+06,NaN
1756,2019-10-23,1289966569,2019-03-12,03676B102,1433270,Antero Midstream Corp,ANTERO RESOURCES Corp,1623925,13D,2019-09-06,...,158419937.0,158419937.0,31.25,1.199239e+06,1.199239e+06,1.199128e+06,-4.076306e+05,-4.076306e+05,-4.075199e+05,-407630.60309
1757,2023-12-29,1289966569,2019-03-12,03676B102,1433270,Antero Midstream Corp,ANTERO RESOURCES Corp,1623925,13D,2020-05-06,...,139042345.0,139042345.0,29.18,1.742201e+06,1.742201e+06,1.753953e+06,5.074137e+06,5.074137e+06,5.062385e+06,NaN


In [22]:
nsv_pair_data.to_csv('../17_data_new/nsv_data')